In [227]:
import os
import pandas as pd

# Specify the target date
target_date_str = "2018-11-26"
target_date = pd.to_datetime(target_date_str)

# Construct the file name (assuming pattern "post_despacho_YYYYMMDD.parquet")
filename = f"post_despacho_{target_date.strftime('%Y%m%d')}.parquet"

# Define the raw folder path (adjust if necessary)
raw_folder = r"..\data\raw"
file_path = os.path.join(raw_folder, filename)

# Read the parquet file
df = pd.read_parquet(file_path)
print("Original DataFrame preview:")
df.head(3)


Original DataFrame preview:


,GRUPOS,INDICE,GRUPO,EMPRESA,CENTRAL,FECHA,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10,H11,H12,H13,H14,H15,H16,H17,H18,H19,H20,H21,H22,H23,H24
0,1 - Térmica,1,Térmica,AES ANDRES,AES ANDRES,2018-11-26T00:00:00,148.01,150.0,147.89,150.0,147.8,150.0,147.8,150.0,148.1,150.0,150.0,150.0,150.0,150.0,150.0,150.0,150.0,148.02,150.0,148.16,150.0,148.14,150.0,150.0
1,1 - Térmica,1,Térmica,BERSAL,BERSAL,2018-11-26T00:00:00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.00,16.0,16.00,16.0,16.00,16.0,16.0
2,1 - Térmica,1,Térmica,CDEEE,RIO SAN JUAN,2018-11-26T00:00:00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0


In [228]:
plant = 'Total Hidroeléctrica'

In [231]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)
df[df['CENTRAL'] == plant].head()

,GRUPOS,INDICE,GRUPO,EMPRESA,CENTRAL,FECHA,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10,H11,H12,H13,H14,H15,H16,H17,H18,H19,H20,H21,H22,H23,H24
98,5 - Totales,5,Totales,Total,Total Hidroeléctrica,2018-11-26T00:00:00,274.71,264.95,146.1,121.35,82.95,74.55,84.94,108.45,195.18,198.45,198.45,168.15,123.05,74.55,74.75,74.75,74.75,81.76,97.15,258.48,279.25,293.25,279.25,234.35


In [225]:
# Define the range of columns to set to zero (from H10 to H15)
cols_to_zero = [f"H{i}" for i in range(1, 25)]

# Update the DataFrame: For rows where CENTRAL is "ESTRELLA DEL MAR 2 SFO", set the specified columns to 0.0
df.loc[df['CENTRAL'] == plant, cols_to_zero] = 0.0


In [230]:
mask = (df['CENTRAL'] == plant)

# Set the value in H20 to zero for the matching ro
df.loc[mask, 'H22'] = 293.25


In [232]:
# Save the modified DataFrame back to the same location in the raw folder
df.to_parquet(file_path, index=False)
print(f"Modified DataFrame saved back to: {file_path}")

Modified DataFrame saved back to: ..\data\raw\post_despacho_20181126.parquet


### Funcion para cambiar si hay error de punto decimal

In [81]:
import os
import glob
import pandas as pd

# --- Parameters ---
plant_name = "inca km22"  # Target plant name (assumed normalized: lowercase, no accents)
raw_folder = r"..\data\raw"  # Path to the raw folder

# Dictionary of target dates and corresponding max_gen values
target_values = {
    "2016-09-14": 97.5
}

# Convert target dates to filename date format (YYYYMMDD)
target_file_dates = {pd.to_datetime(d).strftime("%Y%m%d") for d in target_values.keys()}

# --- Identify all Parquet files in the raw folder that are of interest ---
all_files = glob.glob(os.path.join(raw_folder, "*.parquet"))
target_files = []
for file in all_files:
    base = os.path.basename(file)
    # Expecting pattern: "post_despacho_YYYYMMDD.parquet"
    parts = base.split('_')
    if len(parts) < 3:
        continue
    # Extract date part from the filename (remove extension)
    date_str = parts[-1].split('.')[0]
    if date_str in target_file_dates:
        target_files.append(file)

if not target_files:
    print("No target files found in the raw folder.")
else:
    # Determine hourly columns from one sample file
    sample_df = pd.read_parquet(target_files[0])
    hour_cols = [f"H{i}" for i in range(1, 25) if f"H{i}" in sample_df.columns]
    
    # Process each target file
    for file_path in target_files:
        df = pd.read_parquet(file_path)
        
        # Ensure FECHA is datetime and normalized
        df['FECHA'] = pd.to_datetime(df['FECHA']).dt.normalize()
        # Create helper column for date comparison in "YYYY-MM-DD" format
        df['FECHA_str'] = df['FECHA'].dt.strftime('%Y-%m-%d')
        
        # Loop over each target date in the dictionary
        for target_date, target_value in target_values.items():
            # Create mask: rows where FECHA_str equals target_date and CENTRAL equals plant_name
            mask = (df['FECHA_str'] == target_date) & (df['CENTRAL'].str.lower() == plant_name)
            if mask.any():
                print(f"Processing file '{os.path.basename(file_path)}' for plant '{plant_name}' on date {target_date}.")
                for col in hour_cols:
                    # Divide value by 10 if it equals the target value (within a small tolerance)
                    df.loc[mask, col] = df.loc[mask, col].apply(lambda x: x / 10 if abs(x - target_value) < 1e-6 else x)
        
        # Drop the helper column and save the modified file back to the same location
        df.drop(columns=['FECHA_str'], inplace=True)
        df.to_parquet(file_path, index=False)
        print(f"File '{os.path.basename(file_path)}' updated.\n")

Processing file 'post_despacho_20160914.parquet' for plant 'inca km22' on date 2016-09-14.
File 'post_despacho_20160914.parquet' updated.



#### Función para verificar todas las plantas por fechas

In [233]:
import os
import glob
import pandas as pd

def get_target_los_mina_data(raw_folder, target_dates, target_units):
    """
    Reads all Parquet files from the raw folder and returns a DataFrame containing
    only the rows that match the target dates (in 'YYYY-MM-DD' format) and target units.
    
    If the date column is not named 'FECHA', it tries 'fecha'. If neither is found,
    the file is skipped.
    
    Parameters:
      raw_folder (str): Path to the folder containing raw Parquet files.
      target_dates (list of str): List of target dates in 'YYYY-MM-DD' format.
      target_units (list of str): List of target unit names (assumed normalized to lowercase).
    
    Returns:
      pd.DataFrame: A concatenated DataFrame with matching rows from all files.
    """
    parquet_files = glob.glob(os.path.join(raw_folder, "*.parquet"))
    frames = []
    
    for file_path in parquet_files:
        df = pd.read_parquet(file_path)
        
        # Check if the date column exists; if not, try 'fecha'
        if 'FECHA' in df.columns:
            date_col = 'FECHA'
        elif 'fecha' in df.columns:
            date_col = 'fecha'
            # Rename to standardize
            df.rename(columns={'fecha': 'FECHA'}, inplace=True)
            date_col = 'FECHA'
        else:
            print(f"File {os.path.basename(file_path)} does not have a date column ('FECHA'). Columns: {df.columns}")
            continue
        
        # Ensure FECHA is datetime and normalize it (set time to midnight)
        df['FECHA'] = pd.to_datetime(df['FECHA']).dt.normalize()
        # Create a helper column for date comparison (YYYY-MM-DD)
        df['FECHA_str'] = df['FECHA'].dt.strftime('%Y-%m-%d')
        
        # Filter rows by target dates and target units (assuming CENTRAL is already normalized)
        mask = df['FECHA_str'].isin(target_dates) & df['CENTRAL'].str.lower().isin(target_units)
        if mask.any():
            frames.append(df[mask].copy())
    
    if frames:
        result = pd.concat(frames, ignore_index=True)
        result.drop(columns=['FECHA_str'], inplace=True)
        return result
    else:
        return pd.DataFrame()

# --- Parameters ---
raw_folder = r"..\data\raw"
target_dates = [
    "2017-05-25",
    "2017-08-19",
    "2017-09-04",
    "2017-09-05",
    "2018-03-08",
    "2018-05-10",
    "2018-05-21",
    "2018-06-02",
    "2018-06-03",
    "2018-06-04",
    "2018-06-05",
    "2018-06-06",
    "2018-06-07",
    "2018-06-08",
    "2018-06-09",
    "2018-06-10",
    "2018-06-12",
    "2018-06-14",
    "2018-07-01",
    "2018-07-02",
    "2018-07-03",
    "2018-08-01",
    "2018-08-02",
    "2018-10-06",
    "2018-10-07",
    "2018-10-08",
    "2018-10-09"
]


target_units = [
        "sabana yegua"
    ]
# --- Call the function and display the resulting DataFrame ---
df_target = get_target_los_mina_data(raw_folder, target_dates, target_units)

print("Target data for the specified dates and plants:")
display(df_target)

File post_despacho_20160221.parquet does not have a date column ('FECHA'). Columns: Index([], dtype='object')
File post_despacho_20160507.parquet does not have a date column ('FECHA'). Columns: Index([], dtype='object')
File post_despacho_20160508.parquet does not have a date column ('FECHA'). Columns: Index([], dtype='object')
File post_despacho_20160509.parquet does not have a date column ('FECHA'). Columns: Index([], dtype='object')
File post_despacho_20240602.parquet does not have a date column ('FECHA'). Columns: Index([], dtype='object')
File post_despacho_20241201.parquet does not have a date column ('FECHA'). Columns: Index([], dtype='object')
Target data for the specified dates and plants:


,GRUPOS,INDICE,GRUPO,EMPRESA,CENTRAL,FECHA,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10,H11,H12,H13,H14,H15,H16,H17,H18,H19,H20,H21,H22,H23,H24
0,3 - Hidroeléctrica,3,Hidroeléctrica,EGEHID,SABANA YEGUA,2017-05-25,11.9,12.1,12.45,12.2,12.00,11.88,11.56,11.97,11.96,11.73,11.40,12.10,12.45,11.96,12.46,12.30,11.50,11.72,12.36,12.15,25.00,11.98,12.11,12.1
1,3 - Hidroeléctrica,3,Hidroeléctrica,EGEHID,SABANA YEGUA,2017-08-19,9.5,9.6,9.60,10.2,9.94,30.26,9.60,10.38,10.50,10.10,9.93,10.52,10.10,10.00,10.55,9.55,9.20,8.91,7.96,7.80,9.86,10.33,10.03,10.1
2,3 - Hidroeléctrica,3,Hidroeléctrica,EGEHID,SABANA YEGUA,2017-09-04,10.4,10.2,9.80,10.0,9.90,9.80,9.80,10.30,10.40,10.34,10.34,10.30,10.28,10.28,10.49,10.49,30.81,10.49,0.00,10.49,0.00,10.49,20.09,10.4
3,3 - Hidroeléctrica,3,Hidroeléctrica,EGEHID,SABANA YEGUA,2017-09-05,10.2,10.4,10.20,10.3,9.70,30.43,10.61,31.02,10.40,10.50,10.50,10.31,29.22,10.31,10.75,30.08,9.85,29.67,10.78,10.07,28.87,9.92,9.95,10.4
4,3 - Hidroeléctrica,3,Hidroeléctrica,EGEHID,SABANA YEGUA,2018-03-08,11.7,11.7,11.90,12.0,11.66,11.66,11.75,11.75,11.50,29.84,12.70,11.75,11.75,11.75,11.75,10.90,11.75,11.75,11.75,11.75,11.75,11.75,11.75,0.0
5,3 - Hidroeléctrica,3,Hidroeléctrica,EGEHID,SABANA YEGUA,2018-05-10,11.2,11.2,11.20,11.2,11.05,11.11,11.10,30.00,11.20,11.10,11.20,11.20,11.20,11.20,11.20,11.15,11.00,11.20,11.08,11.10,11.05,11.04,11.10,11.2
6,3 - Hidroeléctrica,3,Hidroeléctrica,EGEHID,SABANA YEGUA,2018-05-21,11.2,11.2,11.20,11.2,10.87,11.20,11.20,30.00,11.20,0.00,0.00,0.00,10.87,10.00,10.50,10.87,10.80,10.80,10.80,10.80,9.10,10.40,10.00,9.1
7,3 - Hidroeléctrica,3,Hidroeléctrica,EGEHID,SABANA YEGUA,2018-06-02,10.4,10.5,10.70,10.5,10.15,10.15,9.90,10.08,9.47,10.30,10.04,30.00,30.00,10.13,10.21,9.91,9.66,10.20,9.66,10.46,10.00,10.16,30.00,10.3
8,3 - Hidroeléctrica,3,Hidroeléctrica,EGEHID,SABANA YEGUA,2018-06-03,10.3,10.5,10.10,10.2,10.33,10.33,10.33,30.00,10.33,10.33,10.33,10.33,10.33,10.33,10.33,10.33,30.00,10.21,10.21,10.21,10.21,10.21,10.21,10.3
9,3 - Hidroeléctrica,3,Hidroeléctrica,EGEHID,SABANA YEGUA,2018-06-04,10.7,10.4,10.10,10.5,10.40,10.21,30.00,10.21,10.21,10.21,10.21,10.21,30.00,10.21,10.21,10.21,10.21,10.21,10.21,29.08,10.21,10.21,10.21,10.3


In [195]:
df_target.to_csv('limpieza quisqueya 1.csv')

#### Cambios de valores con el maximo de la planta

In [245]:
import os
import glob
import pandas as pd

# --- Parameters ---
plant_name = "sabana yegua"  # Target plant (normalized: lowercase, no accents)
raw_folder = r"..\data\raw"   # Path to the raw folder

# Dictionary of target dates and their original max_gen values (to be replaced)
target_values = {
    "2018-09-09": 29.39
}

# New value to set: 13 mega
new_value = 13

# Convert target dates to filename date format (YYYYMMDD)
target_file_dates = {pd.to_datetime(d).strftime("%Y%m%d") for d in target_values.keys()}

# --- Identify all Parquet files in the raw folder that are of interest ---
all_files = glob.glob(os.path.join(raw_folder, "*.parquet"))
target_files = []
for file in all_files:
    base = os.path.basename(file)
    # Expected filename pattern: "post_despacho_YYYYMMDD.parquet"
    parts = base.split('_')
    if len(parts) < 3:
        continue
    date_str = parts[-1].split('.')[0]  # Extract date part from the filename
    if date_str in target_file_dates:
        target_files.append(file)

if not target_files:
    print("No target files found in the raw folder.")
else:
    # Determine hourly columns from one sample file
    sample_df = pd.read_parquet(target_files[0])
    hour_cols = [f"H{i}" for i in range(1, 25) if f"H{i}" in sample_df.columns]
    
    # Process each target file
    for file_path in target_files:
        df = pd.read_parquet(file_path)
        
        # Ensure FECHA is datetime and normalized
        df['FECHA'] = pd.to_datetime(df['FECHA']).dt.normalize()
        # Create helper column for date comparison in "YYYY-MM-DD" format
        df['FECHA_str'] = df['FECHA'].dt.strftime('%Y-%m-%d')
        
        # Loop over each target date and its original max_gen value
        for target_date, original_value in target_values.items():
            # Create mask for rows with matching FECHA_str and plant name (case-insensitive)
            mask = (df['FECHA_str'] == target_date) & (df['CENTRAL'].str.lower() == plant_name)
            if mask.any():
                print(f"Processing file '{os.path.basename(file_path)}' for plant '{plant_name}' on date {target_date}.")
                for col in hour_cols:
                    # Replace the value with new_value (13 mega) if it matches the original value (within tolerance)
                    df.loc[mask, col] = df.loc[mask, col].apply(lambda x: new_value if abs(x - original_value) < 1e-6 else x)
        
        # Drop the helper column and save the modified file back to the same location
        df.drop(columns=['FECHA_str'], inplace=True)
        df.to_parquet(file_path, index=False)
        print(f"File '{os.path.basename(file_path)}' updated.\n")

Processing file 'post_despacho_20180909.parquet' for plant 'sabana yegua' on date 2018-09-09.
File 'post_despacho_20180909.parquet' updated.

